# Module 3 - LLM Fundamentals Assessment (Instructor Version)

**INSTRUCTOR / GRADING TEMPLATE**

This notebook contains **hidden assessment logic** and must NOT be shared with students.

Purpose:
- Inject student code programmatically
- Run automated tests
- Produce authoritative scores

In [ ]:
# === HIDDEN: SCORING SETUP ===
__assessment_scores = {}
__assessment_feedback = {}

def record_score(exercise, points, max_points, feedback=None):
    __assessment_scores[exercise] = (points, max_points)
    if feedback:
        __assessment_feedback[exercise] = feedback

In [ ]:
# === GRADING ENVIRONMENT CONFIG ===
# Mock LLM for grading - no actual LLM available on GitHub Actions

LLM_BASE_URL = "http://mock-llm-for-grading"
LLM_API_KEY = None
DEFAULT_MODEL = "phi3:mini"

import requests
import json
import re

# === MOCK LLM SETUP ===
# We inject a mock requests.post that returns predictable responses
# This allows testing student code structure without a real LLM

_original_post = requests.post
_mock_call_count = 0

def _mock_post(url, **kwargs):
    """Mock requests.post for LLM endpoints."""
    global _mock_call_count
    _mock_call_count += 1
    
    # Check if this is an LLM call
    if '/api/chat' in url or '/chat/direct' in url:
        payload = kwargs.get('json', {})
        messages = payload.get('messages', [])
        prompt = messages[0].get('content', '') if messages else ''
        temperature = payload.get('temperature', 0.0)
        
        # Generate predictable mock responses based on prompt content
        if 'json' in prompt.lower() or '{' in prompt:
            # Return JSON-like response (sometimes wrapped in markdown)
            if _mock_call_count % 2 == 0:
                response_text = '```json\n{"test": 1, "value": 42}\n```'
            else:
                response_text = '{"test": 1, "value": 42}'
        elif 'hello' in prompt.lower():
            response_text = "Hello!"
        elif 'color' in prompt.lower() or 'fruit' in prompt.lower():
            if temperature > 0.5:
                response_text = "Red, Blue, Green (high temp response)"
            else:
                response_text = "Red, Blue, Green"
        else:
            response_text = f"Mock LLM response for: {prompt[:50]}..."
        
        # Create mock response object
        class MockResponse:
            status_code = 200
            def raise_for_status(self):
                pass
            def json(self):
                return {"message": {"content": response_text}}
        
        return MockResponse()
    
    # For non-LLM calls, use original requests.post
    return _original_post(url, **kwargs)

# Monkey-patch requests.post
requests.post = _mock_post

## Exercise 1 - Basic LLM Caller

In [ ]:
# === HIDDEN TEST: Exercise 1 ===
points = 0
feedback = []

try:
    assert 'call_llm' in dir() or 'call_llm' in globals(), "Function 'call_llm' not defined"
    assert callable(call_llm), "'call_llm' should be a function"
    points += 1
    feedback.append("✓ Function 'call_llm' defined")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking function: {type(e).__name__}")

try:
    result = call_llm("Say hello in one word.")
    assert isinstance(result, str), f"Should return str, got {type(result).__name__}"
    points += 1
    feedback.append("✓ Function returns a string")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error calling function: {type(e).__name__}: {e}")

try:
    result = call_llm("Say hello in one word.")
    assert len(result) > 0, "Response should not be empty"
    points += 1
    feedback.append(f"✓ Response contains text ({len(result)} chars)")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking response: {type(e).__name__}")

try:
    result = call_llm("Say hello in one word.", temperature=0.5)
    assert isinstance(result, str), "Should work with temperature parameter"
    points += 1
    feedback.append("✓ Temperature parameter works")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error with temperature param: {type(e).__name__}")

record_score('Exercise 1', points, 4, feedback)

## Exercise 2 - Extract Response Text

In [ ]:
# === HIDDEN TEST: Exercise 2 ===
points = 0
feedback = []

try:
    assert 'get_response_text' in dir() or 'get_response_text' in globals(), "Function 'get_response_text' not defined"
    assert callable(get_response_text), "'get_response_text' should be a function"
    points += 1
    feedback.append("✓ Function 'get_response_text' defined")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking function: {type(e).__name__}")

try:
    result = get_response_text("Say hello")
    assert isinstance(result, str), f"Should return str, got {type(result).__name__}"
    points += 1
    feedback.append("✓ Function returns a string")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error calling function: {type(e).__name__}")

try:
    result = get_response_text("Say hello")
    assert len(result) > 0, "Response should not be empty"
    points += 1
    feedback.append(f"✓ Function returns non-empty text ({len(result)} chars)")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking response: {type(e).__name__}")

record_score('Exercise 2', points, 3, feedback)

## Exercise 3 - JSON Output Parser

In [ ]:
# === HIDDEN TEST: Exercise 3 ===
points = 0
feedback = []

try:
    assert 'parse_json_response' in dir() or 'parse_json_response' in globals(), "Function 'parse_json_response' not defined"
    assert callable(parse_json_response), "'parse_json_response' should be a function"
    points += 1
    feedback.append("✓ Function 'parse_json_response' defined")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking function: {type(e).__name__}")

try:
    result = parse_json_response('Return ONLY this JSON: {"test": 1}')
    assert isinstance(result, tuple), f"Should return tuple, got {type(result).__name__}"
    assert len(result) == 2, f"Tuple should have 2 elements, got {len(result)}"
    points += 1
    feedback.append("✓ Function returns a tuple with 2 elements")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error calling function: {type(e).__name__}")

try:
    success, result = parse_json_response('Return ONLY this JSON: {"test": 1}')
    assert isinstance(success, bool), f"First element should be bool, got {type(success).__name__}"
    points += 1
    feedback.append("✓ First tuple element is a boolean")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking tuple: {type(e).__name__}")

try:
    success, result = parse_json_response('Return ONLY this exact JSON, no other text: {"value": 42}')
    if success:
        assert isinstance(result, dict), "On success, second element should be dict"
        points += 1
        feedback.append("✓ Correctly parses JSON response (handles markdown wrapping)")
    else:
        points += 0.5
        feedback.append(f"✗ JSON parsing failed: {result}")
except Exception as e:
    feedback.append(f"✗ Error in JSON test: {type(e).__name__}")

record_score('Exercise 3', points, 4, feedback)

## Exercise 4 - Temperature Comparison

In [ ]:
# === HIDDEN TEST: Exercise 4 ===
points = 0
feedback = []

try:
    assert 'compare_temperatures' in dir() or 'compare_temperatures' in globals(), "Function 'compare_temperatures' not defined"
    assert callable(compare_temperatures), "'compare_temperatures' should be a function"
    points += 1
    feedback.append("✓ Function 'compare_temperatures' defined")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking function: {type(e).__name__}")

try:
    result = compare_temperatures("List 3 colors.")
    assert isinstance(result, dict), f"Should return dict, got {type(result).__name__}"
    points += 1
    feedback.append("✓ Function returns a dictionary")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error calling function: {type(e).__name__}: {e}")

try:
    result = compare_temperatures("List 3 colors.")
    assert 'low_temp' in result, "Dict should have 'low_temp' key"
    assert 'high_temp' in result, "Dict should have 'high_temp' key"
    assert 'are_identical' in result, "Dict should have 'are_identical' key"
    points += 1
    feedback.append("✓ Dictionary has all required keys")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking keys: {type(e).__name__}")

try:
    result = compare_temperatures("List 3 colors.")
    assert isinstance(result['are_identical'], bool), "'are_identical' should be boolean"
    assert isinstance(result['low_temp'], str), "'low_temp' should be string"
    assert isinstance(result['high_temp'], str), "'high_temp' should be string"
    points += 1
    feedback.append("✓ All values have correct types")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking types: {type(e).__name__}")

record_score('Exercise 4', points, 4, feedback)

## Exercise 5 - Structured Prompt Builder

In [ ]:
# === HIDDEN TEST: Exercise 5 ===
points = 0
feedback = []

try:
    assert 'build_structured_prompt' in dir() or 'build_structured_prompt' in globals(), "Function 'build_structured_prompt' not defined"
    assert callable(build_structured_prompt), "'build_structured_prompt' should be a function"
    points += 1
    feedback.append("✓ Function 'build_structured_prompt' defined")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking function: {type(e).__name__}")

try:
    result = build_structured_prompt("Be helpful", "Explain X", ["Keep it short"])
    assert isinstance(result, str), f"Should return str, got {type(result).__name__}"
    points += 1
    feedback.append("✓ Function returns a string")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error calling function: {type(e).__name__}")

try:
    result = build_structured_prompt("Be helpful", "Explain X", ["Keep it short"])
    assert 'SYSTEM:' in result, "Should contain 'SYSTEM:' label"
    assert 'TASK:' in result, "Should contain 'TASK:' label"
    assert 'CONSTRAINTS:' in result, "Should contain 'CONSTRAINTS:' label"
    points += 1
    feedback.append("✓ Prompt contains all section labels")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking labels: {type(e).__name__}")

try:
    result = build_structured_prompt("Be helpful", "Explain X", ["Short", "Clear"])
    assert '- Short' in result or '- short' in result.lower(), "Constraints should be prefixed with '- '"
    assert '- Clear' in result or '- clear' in result.lower(), "Each constraint should have '- ' prefix"
    points += 1
    feedback.append("✓ Constraints are properly formatted with '- ' prefix")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking constraints: {type(e).__name__}")

try:
    result = build_structured_prompt("Test system", "Test task", ["C1", "C2"])
    assert 'Test system' in result, "System instruction should be in output"
    assert 'Test task' in result, "Task should be in output"
    points += 1
    feedback.append("✓ All inputs are included in output")
except AssertionError as e:
    feedback.append(f"✗ {e}")
except Exception as e:
    feedback.append(f"✗ Error checking content: {type(e).__name__}")

record_score('Exercise 5', points, 5, feedback)

In [ ]:
# === HIDDEN: WRITE RESULTS ===
import json
import datetime

result = {
    'scores': __assessment_scores,
    'feedback': __assessment_feedback,
    'timestamp': datetime.datetime.now().isoformat()
}

with open('assessment_result.json', 'w') as f:
    json.dump(result, f, indent=2)

print("Assessment Results:")
total = sum(s[0] for s in __assessment_scores.values())
max_total = sum(s[1] for s in __assessment_scores.values())
for exercise, (pts, max_pts) in __assessment_scores.items():
    print(f"  {exercise}: {pts}/{max_pts}")
    if exercise in __assessment_feedback:
        for fb in __assessment_feedback[exercise]:
            print(f"    {fb}")
print(f"\nTotal: {total}/{max_total}")